# CNN Classification

***


# Getting Started

In [1]:
import time
import json

import numpy as np
import tensorflow as tf
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from src.repository import BarsRepo, DbLocation, DbSample

In [2]:
# image_dir = Path('../../data/charts/5min')

# Create File DataFrame

In [3]:
N_AFTER_BARS = 5  # 5 / 0
CHOSEN_N = 12  # from gap
PRICE_DIFF_FROM = 0  # 5 / 0

CHOSEN_N = CHOSEN_N - PRICE_DIFF_FROM  # FROM last bar in picture


bars_repo = BarsRepo(DbLocation.LOCAL, DbSample.ALL)
charts_df = bars_repo.get_charts_news_nth_price(n_after_bars=N_AFTER_BARS)

In [4]:
charts_df = charts_df.sample(frac=1.0, random_state=1).reset_index(drop=True)
charts_df[f'n_{CHOSEN_N}'] = charts_df.apply(lambda row : str(int(json.loads(row[f'n_after_{PRICE_DIFF_FROM}'])[f'{CHOSEN_N}'] >= 0)), axis = 1)
charts_df['filepath_jupyter'] = charts_df.apply(lambda row : f'../../{row["filepath"]}', axis = 1)

In [5]:
train_df, test_df = train_test_split(charts_df, train_size=0.7, shuffle=True, random_state=1)

In [ ]:
print(f"gaps with y value = 1: {charts_df[f'n_{CHOSEN_N}'].tolist().count('1')}")
print(f"gaps with y value = 0: {charts_df[f'n_{CHOSEN_N}'].tolist().count('0')}")

# Loading Images

In [6]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [ ]:
train_df

In [ ]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='filepath_jupyter',
    y_col=f'n_{CHOSEN_N}',
    target_size=(119, 86),  # (119, 86), (714, 516)
    class_mode='binary',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='filepath_jupyter',
    y_col=f'n_{CHOSEN_N}',
    target_size=(119, 86),  # (119, 86), (714, 516)
    class_mode='binary',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='filepath_jupyter',
    y_col=f'n_{CHOSEN_N}',
    target_size=(119, 86),  # (119, 86), (714, 516)
    class_mode='binary',
    batch_size=32,
    shuffle=False
)

# Training

In [ ]:
inputs = tf.keras.Input(shape=(119, 86, 3))  # (714, 516, 3) / (119, 86, 3)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu')(inputs)
x = tf.keras.layers.MaxPool2D(pool_size=2)(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=2)(x)
x = tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=2)(x)

x = tf.keras.layers.Flatten()(x)

x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
start = time.time()

def get_f1(y_true, y_pred):
    import tensorflow.keras.backend as K
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val


optimizer = tf.keras.optimizers.Adam(lr=0.000_1)
model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy', 'AUC', get_f1]
)

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=50,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True,
            verbose=1
        )
    ]
)

end = time.time()
print(f'training TIME: {end - start}')

# Results

In [ ]:
predicted = np.squeeze(model.predict(test_images))
actual = test_images.labels

test_loss, test_acc, test_auc, test_f1 = model.evaluate(test_images)
print(f"Test loss: {test_loss:.3f}")
print(f"Test accuracy: {test_acc:.3f}")
print(f"Test AUC: {test_auc:.3f}")
print(f"Test F1-score: {test_f1:.3f}")

r2 = r2_score(actual, predicted)
print("Test R^2 Score: {:.5f}".format(r2))

print(f' - loss: **{test_loss:.3f}**; acc: **{test_acc:.3f}**; AUC: **{test_auc:.3f}**; F1: **{test_f1:.3f}**; R2: **{r2:.5f}**')

In [ ]:
def m2tex(model, modelName):
    stringlist = []
    model.summary(line_length=70, print_fn=lambda x: stringlist.append(x))
    del stringlist[1:-4:2]
    del stringlist[-1]
    for ix in range(1, len(stringlist) - 3):
        tmp = stringlist[ix]
        stringlist[ix] = tmp[0:31] + "& " + tmp[31:59] + "& " + tmp[59:] + "\\\\ \hline"
    stringlist[0] = "Model: {} \\\\ \hline".format(modelName)
    stringlist[1] = stringlist[1] + " \hline"
    stringlist[-4] += " \hline"
    stringlist[-3] += " \\\\"
    stringlist[-2] += " \\\\"
    stringlist[-1] += " \\\\ \hline"
    prefix = ["\\begin{table}[]", "\\begin{tabular}{lll}"]
    suffix = ["\end{tabular}", "\caption{{Model summary for {}.}}".format(modelName), "\label{tab:model-summary}",
              "\end{table}"]
    stringlist = prefix + stringlist + suffix
    out_str = " \n".join(stringlist)
    out_str = out_str.replace("_", "\_")
    out_str = out_str.replace("#", "\#")
    print(out_str)


m2tex(model, 'OLA')

In [ ]:
import matplotlib.pyplot as plt
accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()